<a href="https://colab.research.google.com/github/Athul-nambiar/Fine-Tuning-/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "aboonaji/llama2finetune-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Corrected BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True  # ✅ Correct setting for CPU offloading
)

# Corrected Model Loading with Custom Device Map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # ✅ Auto-distributes model across CPU/GPU
)

print("✅ Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

✅ Model loaded successfully!


In [3]:
import torch

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"


# Define your prompt
prompt = "can you give me about stone man syndrome?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=250,  # Adjust as needed
        temperature=0.5,  # Controls randomness (lower = more focused)
        top_p=1,  # Nucleus sampling (higher = more diverse output)
        repetition_penalty=1.1,  # Reduce repetitive text
    )

# Decode and print result
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


can you give me about stone man syndrome?

Stone Man Syndrome, also known as Fibrodysplasia Ossificans Progressiva (FOP), is a rare genetic disorder that affects the body's musculoskeletal system. It is characterized by the abnormal growth of bone tissue in soft tissues such as muscles, tendons, and ligaments.

The symptoms of Stone Man Syndrome typically become apparent during childhood or adolescence and progress slowly over time. The main features of the condition include:

1. Bony outgrowths: In FOP, excessive bone growth occurs in soft tissues, leading to the formation of bony outgrowths called heterotopic ossification (HO). These outgrowths can occur anywhere in the body but are most commonly seen in the neck, shoulder, and limbs.
2. Limited mobility: As the bony outgrowths grow, they can restrict movement in the affected areas, leading to limited mobility and stiffness.
3. Pain: The bony outgrowths can cause pain and discom


In [6]:
!pip install datasets

from datasets import load_dataset

# Load the dataset
ds = load_dataset("aboonaji/wiki_medical_terms_llam2_format")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00


wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

In [7]:
# Check available splits
print(ds.keys())

# Access the training split
train_dataset = ds["train"]


dict_keys(['train'])


In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Access the 'train' split from the dataset
train_data = ds["train"]

# Convert to pandas DataFrame for easier manipulation
train_df = train_data.to_pandas()

# First split into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Now split the train_df into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert back to datasets.Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Check the splits
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 4390
Validation dataset size: 1098
Test dataset size: 1373


In [10]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = ds["train"]

# Convert to pandas DataFrame for easier manipulation
train_df = train_data.to_pandas()

# First split into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Now split the train_df into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert back to datasets.Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the tokenizer (choose the one based on your model)
tokenizer = AutoTokenizer.from_pretrained('aboonaji/llama2finetune-v2')

# Define the preprocess function for tokenization
def preprocess_function(examples):
    # Assuming the 'text' field contains the raw input text
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)

# Apply preprocessing to the training dataset
train_tokenized_ds = train_dataset.map(preprocess_function, batched=True)

# Apply preprocessing to the validation dataset
val_tokenized_ds = val_dataset.map(preprocess_function, batched=True)

# Apply preprocessing to the test dataset
test_tokenized_ds = test_dataset.map(preprocess_function, batched=True)

# Check a sample from the tokenized training dataset to verify
print(train_tokenized_ds[0])  # Prints the first tokenized example from training data

# Optionally, check a sample from the validation dataset
print(val_tokenized_ds[0])  # Prints the first tokenized example from validation data

# Optionally, check a sample from the test dataset
print(test_tokenized_ds[0])  # Prints the first tokenized example from test data


Map:   0%|          | 0/4390 [00:00<?, ? examples/s]

Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

Map:   0%|          | 0/1373 [00:00<?, ? examples/s]

{'text': "<s> [INST] <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> What is Subclinical infection and explain in detail? [/INST] A subclinical infection — sometimes called a preinfection or inapparent infection — is an infection that, being subclinical, is nearly or completely asymptomatic (no signs or symptoms). A subclinically infected person is thus a paucisymptomatic or asymptomatic carrier of a microbe, intestinal parasite, or virus that usually is a pathogen causing illness, at least in some individuals. Many pa

In [1]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

# Load your pretrained model in a memory-efficient way
model = AutoModelForCausalLM.from_pretrained(
    'aboonaji/llama2finetune-v2',
    device_map='auto',     # Automatically assigns model layers to available devices
    load_in_8bit=True      # Loads the model in 8-bit mode to reduce memory usage (if supported)
)

# Define LoRA parameters
lora_config = LoraConfig(
    r=8,                      # Rank of the LoRA decomposition
    lora_alpha=16,            # Scaling factor for the LoRA updates
    target_modules=["q_proj", "v_proj"],  # Target modules for applying LoRA
    lora_dropout=0.1,         # Dropout probability for LoRA layers
    bias="none"               # LoRA applied only on weights (bias set to "none")
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

print("LoRA configuration has been successfully applied to the model.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LoRA configuration has been successfully applied to the model.


In [2]:
from transformers import TrainingArguments, Trainer

# Define training arguments with memory-saving strategies
training_args = TrainingArguments(
    output_dir='./results',                    # Directory for saving checkpoints/results
    per_device_train_batch_size=2,             # Very small batch size to lower memory usage
    per_device_eval_batch_size=2,              # Small batch size for evaluation
    gradient_accumulation_steps=8,             # Accumulate gradients to simulate a larger batch size
    num_train_epochs=3,                        # Number of epochs; adjust as needed
    fp16=True,                                 # Enable FP16 mixed precision training
    logging_steps=50,                          # Log training metrics every 50 steps
    evaluation_strategy='steps',               # Evaluate during training at regular steps
    eval_steps=50,                             # Evaluate every 50 steps
    save_steps=500,                            # Save a checkpoint every 500 steps
    save_total_limit=2,                        # Limit the total number of checkpoints to reduce disk usage
)

# (Optional) If your model supports it, enable gradient checkpointing to save memory
if hasattr(model, "enable_gradient_checkpointing"):
    model.enable_gradient_checkpointing()

# Initialize the Trainer with your LoRA-wrapped model and tokenized datasets
trainer = Trainer(
    model=model,                               # Your LoRA-configured model
    args=training_args,
    train_dataset=train_tokenized_ds,          # Your tokenized training dataset
    eval_dataset=val_tokenized_ds,             # Your tokenized validation dataset
)

# Start the fine-tuning process
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'train_tokenized_ds' is not defined